In [ ]:
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/viz.html


In [ ]:
import os 
print(os.getcwd())


In [ ]:
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/WN_BPK25")
print(os.getcwd())

In [ ]:
! ls -1 *.mcool

In [ ]:
wn_bpk25_rep12_mcool = "WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_rep1_mcool = "WN_mESC_BPK25_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
wn_bpk25_rep2_mcool = "WN_mESC_BPK25_HiC_rep2.nodups.pairs.1k.zoomify.mcool"

In [ ]:
import cooler
# to print which resolutions are stored in the mcool, use list_coolers
cooler.fileops.list_coolers(wn_bpk25_rep12_mcool)


In [ ]:
### to load a cooler with a specific resolution use the following syntax:
wn_bpk25_rep12_mcool_r20k = cooler.Cooler(f'{wn_bpk25_rep12_mcool}::resolutions/20000')
wn_bpk25_rep1_mcool_r20k = cooler.Cooler(f'{wn_bpk25_rep1_mcool}::resolutions/20000')
wn_bpk25_rep2_mcool_r20k = cooler.Cooler(f'{wn_bpk25_rep2_mcool}::resolutions/20000')


# wn_bpk25_rep12_mcool_r20k raw count plot

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42

chromsome = "chr1"
start = 16_600_000
end = 17_800_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)

fig = plt.figure(figsize=(9, 9))
# fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.93, wspace=0.18)
# fig.suptitle("xxx_plot")

# ax1
ax1 = fig.add_subplot(1, 1, 1)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), vmax=800,  cmap='fall')
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region)*100000, vmax=800,  cmap='fall')
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=False)');


# wn_bpk25_rep12_mcool_r20k blance and multiplied 10^5 as WN Fig 2 A/B description

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42

chromsome = "chr1"
start = 16_600_000
end = 17_800_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)

fig = plt.figure(figsize=(9, 9))
# fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.93, wspace=0.18)
# fig.suptitle("xxx_plot")

# ax1
ax1 = fig.add_subplot(1, 1, 1)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmax=800, cmap='fall')
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_600_000
end = 17_800_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)

fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.99, wspace=0.18)
# fig.suptitle("xxx_plot")

# ax1
ax1 = fig.add_subplot(1, 1, 1)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, 
                 vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'{chromsome}:{start:,}-{end:,}', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.show()


# add Smoothing & Interpolation based on WN description

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.99, wspace=0.18)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'{chromsome}:{start:,}-{end:,}', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.99, wspace=0.18)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'{chromsome}:{start:,}-{end:,}', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.99, wspace=0.18)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'{chromsome}:{start:,}-{end:,}', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0.04, right=0.98, bottom=0.08, top=0.99, wspace=0.18)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'{chromsome}:{start:,}-{end:,}', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.show()


# rep12 merged mcool r20k plot

## r20k Fig 2A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig2A_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig2A_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 2B BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr9"
start = 13_790_000
end = 15_100_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig2B_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig2B_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 3A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr8"
start = 88_500_000
end = 94_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=12)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3A_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3A_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 3C BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr8"
start = 119_000_000
end = 120_700_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=1, max_levels=5)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3C_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3C_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig3E BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 9_800_000
end = 14_600_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3E_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3E_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig5A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chrX"
start = 100_400_000
end = 103_400_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig5A_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig5A_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chrX"
start = 100_470_000
end = 100_660_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=3000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig5A_BPK25.zoomIn.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig5A_BPK25.zoomIn.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig S3I BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr3"
start = 39_900_000
end = 42_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=11)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_FigS3I_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_FigS3I_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


## r20k Fig S3J BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr14"
start =109_400_000
end = 111_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_FigS3J_BPK25.rep12merged.r20k.fall.heatmap.pdf")
plt.savefig("WN_FigS3J_BPK25.rep12merged.r20k.fall.heatmap.png")
plt.show()


# rep1 mcool r20k plot

## r20k Fig 2A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4, max_levels=9)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig2A_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig2A_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 2B BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr9"
start = 13_790_000
end = 15_100_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig2B_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig2B_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 3A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr8"
start = 88_500_000
end = 94_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=12)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3A_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3A_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 3C BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr8"
start = 119_000_000
end = 120_700_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=1, max_levels=5)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3C_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3C_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig3E BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 9_800_000
end = 14_600_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3E_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3E_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig5A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chrX"
start = 100_400_000
end = 103_400_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig5A_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig5A_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chrX"
start = 100_470_000
end = 100_660_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=3000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig5A_BPK25.zoomIn.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig5A_BPK25.zoomIn.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig S3I BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr3"
start = 39_900_000
end = 42_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=11)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_FigS3I_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_FigS3I_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


## r20k Fig S3J BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr14"
start =109_400_000
end = 111_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep1_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep1_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep1_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep1_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep1_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep1_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep1_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_FigS3J_BPK25.rep1.r20k.fall.heatmap.pdf")
plt.savefig("WN_FigS3J_BPK25.rep1.r20k.fall.heatmap.png")
plt.show()


# rep2 mcool r20k plot

## r20k Fig 2A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=4, max_levels=9)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig2A_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig2A_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 2B BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr9"
start = 13_790_000
end = 15_100_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig2B_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig2B_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 3A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr8"
start = 88_500_000
end = 94_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=12)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp, vmax=800, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3A_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3A_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig 3C BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr8"
start = 119_000_000
end = 120_700_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=1, max_levels=5)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3C_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3C_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig3E BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr1"
start = 9_800_000
end = 14_600_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig3E_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig3E_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig5A BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chrX"
start = 100_400_000
end = 103_400_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig5A_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig5A_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chrX"
start = 100_470_000
end = 100_660_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=3000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_Fig5A_BPK25.zoomIn.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_Fig5A_BPK25.zoomIn.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig S3I BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr3"
start = 39_900_000
end = 42_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=11)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_FigS3I_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_FigS3I_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


## r20k Fig S3J BPK25

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

chromsome = "chr14"
start =109_400_000
end = 111_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
wn_bpk25_rep2_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep2_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
wn_bpk25_rep2_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep2_mcool_r20k_adcg)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(wn_bpk25_rep2_mcool_r20k.matrix(balance=True).fetch(plot_region)*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(wn_bpk25_rep2_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'WN_BPK25_rep2_r20k ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

plt.savefig("WN_FigS3J_BPK25.rep2.r20k.fall.heatmap.pdf")
plt.savefig("WN_FigS3J_BPK25.rep2.r20k.fall.heatmap.png")
plt.show()


In [ ]:
import os
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/WN_BPK25")
print(os.getcwd())

In [ ]:
%%bash
mkdir WN_heatmap_replot_under_r20k_results
mv WN*.r20k.*pdf ./WN_heatmap_replot_under_r20k_results/
mv WN*.r20k.*png ./WN_heatmap_replot_under_r20k_results/


In [ ]:
! find . -name WN*.rep1.r20k.*png

In [ ]:
import os
import glob
import platform

from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

os_name = platform.system()
print(os_name)
# 指定目录路径
directory = './WN_heatmap_replot_under_r20k_results/'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, '*rep12*.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
print(len(png_files))
# 打印文件路径列表
for file in png_files:
    # print(file)
    if os_name == 'Windows':
        print(file.split('\\')[2])
    else:
        print(file.split('/')[2])


# add rep12 merged plot to summary ppt

In [ ]:
import os
import glob
import platform

from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

os_name = platform.system()
# 指定目录路径
directory = './WN_heatmap_replot_under_r20k_results/'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, '*rep12*.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
# print(len(png_files))

# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

page_count = int(len(png_files)/2)+1
# print(page_count)
# jpg 图像文件处理序号
pi = 1
for i in range(1, page_count+1):
    top_pi = pi*2-1
    bottom_pi = pi*2
    # print(top_pi)
    # print(bottom_pi)
    slide = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
    # top
    if top_pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(6.80), Cm(0.60), Cm(5.5), Cm(0.5))
        # 设置文本框背景颜色为黄色
        fill = text_box.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[top_pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[top_pi-1].split('\\')[2]
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(5)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[top_pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[top_pi-1], Cm(6.40), Cm(1.2), Cm(6.15), Cm(5.7))
    
        # bottom
    if bottom_pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(6.80), Cm(0.60+7.1), Cm(5.5), Cm(0.5))
        # 设置文本框背景颜色为黄色
        fill = text_box.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[bottom_pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[bottom_pi-1].split('\\')[2]
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(5)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[bottom_pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[bottom_pi-1], Cm(6.40), Cm(1.2+7.1), Cm(6.15), Cm(5.7))

    # 页面索引 +1 
    pi += 1

# 保存 PPT
my_ppt.save('0_WN_BPK25_heatmap_replot_under_r20k_results.summary.pptx')


# add rep1 plot to summary ppt

In [ ]:
import os
import glob
import platform

from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

os_name = platform.system()
# 指定目录路径
directory = './WN_heatmap_replot_under_r20k_results/'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, '*rep1.*.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
# print(len(png_files))

# 打开 PPT 并插入图片
my_ppt = Presentation("0_WN_BPK25_heatmap_replot_under_r20k_results.summary.pptx")

page_count = int(len(png_files)/2)+1
# print(page_count)
# jpg 图像文件处理序号
pi = 1
for i in range(1, page_count+1):
    top_pi = pi*2-1
    bottom_pi = pi*2
    # print(top_pi)
    # print(bottom_pi)
    slide = my_ppt.slides[pi-1]
    # top
    if top_pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(6.80+6.4), Cm(0.60), Cm(5.5), Cm(0.5))
        # 设置文本框背景颜色为黄色
        fill = text_box.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[top_pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[top_pi-1].split('\\')[2]
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(5)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[top_pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[top_pi-1], Cm(6.40+6.4), Cm(1.2), Cm(6.15), Cm(5.7))
    
        # bottom
    if bottom_pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(6.80+6.4), Cm(0.60+7.1), Cm(5.5), Cm(0.5))
        # 设置文本框背景颜色为黄色
        fill = text_box.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[bottom_pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[bottom_pi-1].split('\\')[2]
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(5)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[bottom_pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[bottom_pi-1], Cm(6.40+6.4), Cm(1.2+7.1), Cm(6.15), Cm(5.7))

    # 页面索引 +1 
    pi += 1

# 保存 PPT
my_ppt.save('0_WN_BPK25_heatmap_replot_under_r20k_results.summary.pptx')


# add rep2 plot to summary ppt

In [ ]:
import os
import glob
import platform

from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

os_name = platform.system()
# 指定目录路径
directory = './WN_heatmap_replot_under_r20k_results/'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, '*rep2.*.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
# print(len(png_files))

# 打开 PPT 并插入图片
my_ppt = Presentation("0_WN_BPK25_heatmap_replot_under_r20k_results.summary.pptx")

page_count = int(len(png_files)/2)+1
# print(page_count)
# jpg 图像文件处理序号
pi = 1
for i in range(1, page_count+1):
    top_pi = pi*2-1
    bottom_pi = pi*2
    # print(top_pi)
    # print(bottom_pi)
    slide = my_ppt.slides[pi-1]
    # top
    if top_pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(6.80+12.8), Cm(0.60), Cm(5.5), Cm(0.5))
        # 设置文本框背景颜色为黄色
        fill = text_box.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[top_pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[top_pi-1].split('\\')[2]
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(5)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[top_pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[top_pi-1], Cm(6.40+12.8), Cm(1.2), Cm(6.15), Cm(5.7))
    
        # bottom
    if bottom_pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(6.80+12.8), Cm(0.60+7.1), Cm(5.5), Cm(0.5))
        # 设置文本框背景颜色为黄色
        fill = text_box.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[bottom_pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[bottom_pi-1].split('\\')[2]
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(5)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[bottom_pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[bottom_pi-1], Cm(6.40+12.8), Cm(1.2+7.1), Cm(6.15), Cm(5.7))

    # 页面索引 +1 
    pi += 1

# 保存 PPT
my_ppt.save('0_WN_BPK25_heatmap_replot_under_r20k_results.summary.pptx')